## Решающие деревья

Мы ищем наше решение в виде последовательных логических утверждений - это логический алгоритм классификации

в листе содержится ответ

Во время фита мы строим решающие правила для каждой вершины так, чтобы в вершине преобладал один класс над другим.

В каждой вершине решающее правило имеет вид признак >= Const

- Решающее дерево строится за счет минимизации выбранного критерия после каждого разделения
- Глубина не должна быть слишком большой, чтобы не было оверфита
- Если в лист попал 1 объект - это плохо, хоть это и даст оптимальное значение критерия - это не репрезентативно

У алогритма решающего дерева возможна регуляризация как раз путем ограничения глубины и количества объектов в терминальной вершине

##### Для регрессии:

В листе будет находиться действительное число, а не метка класса

В качестве лосса обычно MSE используют

## Композиции алгоритмов

##### Метод голосования:
есть $a_1,...,a_n$ - обученных алгоритмов, тогда для объекта $x$:
- Классификация: устраиваем голосование и относим $x$ туда, за который больше всего алгоритмов проголосовало
- Регрессия: среднее значение $a_1(x),...,a_n(x)$

#### Bootstrapping:

Надо делить начальную выборку на бутстрепные выборки и потом каждый алгоритм обучать на отдельной бутстрепной выборке

Статистика показывает, что путем создания выборок бустрепом - все они аналогичны исходной

#### Bagging:

https://youtu.be/vqF8wrWjR5s?t=650

Это как раз и есть bootstap + aggregation - обучаем n алгоритмов на n бутстрепных выборок и потом устраиваем голосование

Бэггинг над решающими деревьями - это Случайный лес, это супер популярно на kaggle.com.

Решающие деревья компенсируют свои недостатки в случае бэггинга, так как они умеют восстанавливать довольно сложные зависимости, а благодаря бэггингу случайный лес не так склонен к переобучению

Минусы:
- Слишком долгие вычисления (но можно распараллеливать)

- В индустрии стараются обойтись без композиций

#### Stacking:

1)Обучаем какой-то алгоритм - полученный столбец предсказаний воспринимаем как новый признак, так обучаем несколько алгоритмов и получаем
новый датасет(или добавляем новые признаки к старому датасету), построенный из признаков равных предсказаниям разных алгоритмов.

2)Обучаем какой-то супер крутой метаалгоритм на полученном датасете (а можно и на старом датасете вместе с новыми признаками) и уже получаем итоговый столбец предсказаний

Стэкинг круто использовать как финальный рывок в соревнованиях, потому что он очень тяжеловесный

### Boosting

принимаем решение взвешенным голосованием - то есть каждый следующий алгоритм в цепочке компенсирует ошибки предыдущих

## Градиентный бустинг

Здесь регрессия и классификация рассматривается

Задача - построить алогоритм: функцию $a(x)$, которая оптимизирует loss: $Q(y, a)=\dfrac{1}{l}\sum_{i=1}^lL(y^i, a(x^i))$

Пример для MSE: $L(y, a(x))=(a(x)-y)^2$

Обычно в бустинге используют решающие деревья в качестве базового алгоритма, суть алгоритма: https://youtu.be/JElfEE1OrSU?t=566

Cуть: мы строим алгоритм $a(x)=c_1a_1(x)+...+c_ka_k(x)$, можно взять $c_k = \dfrac{1}{\sqrt{k}}$, все $a_j$ пусть решающие деревья

Строим $a_{k+1}$ просто оптимизируя наш лосс после того как $a_k$ алгоритм построили, то есть для $a_{k+1}$ просто в качестве таргета для $x^i$ объекта(лейбл для обучения алгоритма) берем: $-\dfrac{\partial L(y^i, a)}{\partial a}\vert_{a = a_1(x^i)+...+a_k(x^i)}$. То есть каждый $a_k$ решает свою задачу регрессии со своими таргетами

На словах: построили первое решающее дерево, где вектор таргетов - это просто исходный вектор $y$ и получили некоторые предсказания, потом
строим следующее дерево таким образом, чтобы снизить значение loss общей функции, то есть строим следующее дерево, у которого вектор таргетов - это антиградиент функции потерь с координатами - значениями производной $-\dfrac{\partial L(y^i, a)}{\partial a}\vert_{a = a_1(x^i)+...+a_k(x^i)}$.


Так можно продолжать добавлять $a_k$ в решающий алогоритм и каждый $a_k$ будет решать свою задачу регрессии со своими таргетами, уменьшая лосс

Плюсы и минусы:

Полную ошибку можно разделить на сдвиг(bias) и дисперсию, плюс бустинга в том, что он устраняет bias, а не накапливает его

Бустинг нельзя обучать параллельно

Для бустинга необходимо: число объектов >> числа признаков